In [ ]:
import tweepy
#import pandas as pd
from collections import defaultdict
import re
import json
import requests
import os
from pprint import pprint
from googletrans import Translator, constants
import time
from datetime import datetime

import jmespath
#https://python.land/data-processing/working-with-json/jmespath

1) go through all files in the tweets
	a) get userName from userId
		a.1) check local view, if not in local view query twitter

In [ ]:
load_dotenv()

API_Key = os.environ.get("API_KEY")
API_Key_Secret = os.environ.get("API_KEY_SECRET")
Bearer_Token = os.environ.get("BEARER_TOKEN")


auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
bearer = 'bearer ' + Bearer_Token


In [ ]:
#Initiating Variables -- make sure to manually edit each one

date = datetime.today().strftime('%Y-%m-%d')
dateFolder = date
weekly_filename = f'deduped_weekly_tweets_{date}.json'

In [ ]:
def loadSavedMappings():
    
    AUTHOR_MAP = {}
    
    #author mappings
    map_file = 'authors.json'
    
    fullFilePath = f'twitter_data/saved_mappings/{map_file}'
    with open(fullFilePath, 'r') as f:
        AUTHOR_MAP = json.load(f)
    
    f.close()
    
    return AUTHOR_MAP

AUTHOR_MAP = loadSavedMappings()
print(AUTHOR_MAP)


In [ ]:
#Initiating Varibales -- derived programatically or rarely changed

userName = 'defaultUserName'

#MAPPING_FILE_DIR = 'twitter_data/saved_mappings/'
basepath = f'twitter_data/{dateFolder}/'
baseTweetURL = 'https://twitter.com/{userName}/status/{tweetId}'

AUTHOR_MAP = loadSavedMappings()
    

In [ ]:
def writeSavedMappings(AUTHOR_MAP):
    map_file = 'authors.json'
    
    fullFilePath = f'twitter_data/saved_mappings/{map_file}'
    with open(fullFilePath, 'w') as f:
        f.write(json.dumps(AUTHOR_MAP, indent=4, sort_keys=True))
    
    f.close()
    
writeSavedMappings(AUTHOR_MAP)

In [ ]:
def get_author(author_id):
    
    
    auth_list = jmespath.search(f"data[?id == '{author_id}'].username", AUTHOR_MAP)
    userFields = 'location,url,description,entities'
    where = 'TBD'

    if len(auth_list) == 1:
        where = 'locally'
        author_username = auth_list[0]
    elif len(auth_list) > 1:
        author_username = 'UNKN_MultipleResultsReturned'
    else:
        
        api_getAuthor = f'https://api.twitter.com/2/users?ids={author_id}&user.fields={userFields}'
        response = requests.get(api_getAuthor, headers={"Authorization":bearer})
        author_from_twitter = json.loads(response.text)
        time.sleep(2) #throttle volume
        
        
        if 'data' in author_from_twitter:
            where = 'twitter'
            AUTHOR_MAP['data'].append(author_from_twitter['data'][0])
            writeSavedMappings(AUTHOR_MAP)
            author_username = author_from_twitter['data'][0]['username']
        elif 'errors' in   author_from_twitter:
            author_username = author_from_twitter['errors'][0]['title']
        else:
            author_username = 'UNKNN_NoResultsReturned'
            print(author_from_twitter)

    print(f'found {author_username} : {where} : {author_id}')
    return author_username


#test
author_username = get_author('774273192395280384')




In [ ]:
cnt = 0
iterr = 0
for tweet_json in os.listdir(basepath):
    if os.path.isfile(os.path.join(basepath, tweet_json)):
        

        
        fullFilePath = basepath + tweet_json
        
        iterr +=1
        print(f"working on {iterr}th file: {tweet_json}")
    
        with open(fullFilePath, 'r') as f:
            tweets = json.load(f)
            if tweets['meta']['result_count'] == 0:
                next
            else:
                cnt += tweets['meta']['result_count']
                for tweet in tweets['data']:
                    
                    
                    
                    
                    
                    #print(tweet)
                    author = get_author(jmespath.search("author_id", tweet))
                    #print(tweet_json, ' ')
        f.close()
                

print(cnt , " results processed.")

In [ ]:
#create one giant json with consolidated list

allTweets = {"data" : [], "meta" : {"result_count" : 0}}
tweetDuplicateTracker = {}
tweetDuplicateTracker['totalDupes'] = 0


cnt = 0
iterr = 0
for tweet_json in os.listdir(basepath):
    if os.path.isfile(os.path.join(basepath, tweet_json)):
        

        
        fullFilePath = basepath + tweet_json

        with open(fullFilePath, 'r') as f:
            print(fullFilePath)
            tweets = json.load(f)
            if tweets['meta']['result_count'] == 0:
                next
            else:
                
                cnt += tweets['meta']['result_count']
                for tweet in tweets['data']:

                    
                    tweetId = tweet['id']
                    if tweetId in tweetDuplicateTracker:
                        #dupe
                        tweetDuplicateTracker[tweetId] += 1
                        tweetDuplicateTracker['totalDupes'] += 1
                    else:
                        tweetDuplicateTracker[tweetId] = 0
                        allTweets['data'].append(tweet)
                        allTweets['meta']['result_count'] += 1

        f.close()
                

print(cnt , " results processed. ", allTweets['meta']['result_count'])
fullFilePath = f'twitter_data/weekly_files/{weekly_filename}'
with open(fullFilePath, 'w') as f:
    f.write(json.dumps(allTweets, indent=4, sort_keys=True))
    
    f.close()

In [ ]:
print(tweetDuplicateTracker['totalDupes'])
print(allTweets['meta']['result_count'])

In [ ]:
len(AUTHOR_MAP['data'])